# Separando em Treino e Teste

## Caminhos

In [1]:
import os

#Folder Inicial
path = os.getcwd()

#Subpastas
pathin = path + '\\Entrada\\'
pathfixo = path + '\\Fixo\\'
pathout = path + '\\Saida\\'
pathparcial = path + '\\Parcial\\'
pathaux = path + '\\Auxiliar\\'

## Pacotes

In [2]:
import dill
import pickle
import pandas as pd
pd.set_option('max_colwidth', 3000)

import numpy as np

from time import gmtime, strftime

import re

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV

import unidecode
from unicodedata import normalize

import pygtrie

## Funcoes

In [3]:
with open(pathaux + 'Functions.pickle', 'rb') as f:
    rem_acentos, PreProcess, dicMedian, RemoveNA, dicMeanSd, Scale = dill.load(f)
    #rem_acentos, PreProcess, dicMeanSd, Scale = dill.load(f)

## Lendo a Base

In [4]:
data = pd.read_csv(pathin + 'german_credit.csv', delimiter =',')
print(data.shape)
#DataFrame com quantidade de Linhas
sizes = pd.DataFrame({'descricao': ['Original'], 'linhas': [data.shape[0]]})
sizes

(1010, 21)


,descricao,linhas
0,Original,1010


In [5]:
print(data.shape)
data.head(2)

(1010, 21)


,default,account_check_status,duration_in_month,credit_history,purpose,credit_amount,savings,present_emp_since,installment_as_income_perc,personal_status_sex,...,present_res_since,property,age,other_installment_plans,housing,credits_this_bank,job,people_under_maintenance,telephone,foreign_worker
0,0.0,< 0 DM,6,critical account/ other credits existing (not at this bank),domestic appliances,1169,unknown/ no savings account,.. >= 7 years,4,male : single,...,4,real estate,67,none,own,2,skilled employee / official,1,"yes, registered under the customers name",yes
1,1.0,0 <= ... < 200 DM,48,existing credits paid back duly till now,domestic appliances,5951,... < 100 DM,1 <= ... < 4 years,2,female : divorced/separated/married,...,2,real estate,22,none,own,1,skilled employee / official,1,none,yes


In [6]:
#Resetando index e gerando um id
data.reset_index(drop=True, inplace = True)
data['id'] = data.index

## Ajustando Variavel Resposta

In [7]:
yy = 'default'

print(data[yy].unique())
data[yy].value_counts()

[ 0.  1. nan  5.]


0.0    700
1.0    300
5.0      1
Name: default, dtype: int64

### Elilminar NA

In [8]:
data = data[(data[yy] == 1) | (data[yy] == 0)]

sizes = sizes.append({'descricao': 'Excluindo resposta NA', 'linhas': data.shape[0]}, ignore_index = True)
sizes

,descricao,linhas
0,Original,1010
1,Excluindo resposta NA,1000


In [9]:
print(data[yy].unique())
data[yy].value_counts()

[0. 1.]


0.0    700
1.0    300
Name: default, dtype: int64

### Eliminar Duplicadas

In [10]:
data = data.sort_values(by = ['id'], ascending = False, na_position = 'first')
data.drop_duplicates(subset = ['id'], keep = 'last', inplace = True)

sizes = sizes.append({'descricao': 'Eliminando Ultima Linha Quando id repetido', 'linhas': data.shape[0]}, ignore_index = True)
sizes

,descricao,linhas
0,Original,1010
1,Excluindo resposta NA,1000
2,Eliminando Ultima Linha Quando id repetido,1000


## Separando em Treino e Teste

In [11]:
y = data[yy]
X = data.drop(columns = [yy])

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.25, random_state = 123)

sizes = sizes.append({'descricao': 'Treino', 'linhas': X_train.shape[0]}, ignore_index = True)
sizes = sizes.append({'descricao': 'Teste', 'linhas': X_test.shape[0]}, ignore_index = True)

train0 = pd.merge(X_train, pd.DataFrame(y_train), left_index = True, right_index = True)
teste0 = pd.merge(X_test, pd.DataFrame(y_test), left_index = True, right_index = True)

sizes.to_excel(pathout + 'Tamanhos.xlsx', encoding = 'latin1', index = False)
sizes

,descricao,linhas
0,Original,1010
1,Excluindo resposta NA,1000
2,Eliminando Ultima Linha Quando id repetido,1000
3,Treino,750
4,Teste,250


## Preprocessamento

In [12]:
train1 = PreProcess(dataset = train0, y = 'default')
teste1 = PreProcess(dataset = teste0, y = 'default')

print(train1.shape)
print(train0.shape)
print()
print(teste1.shape)
print(teste0.shape)
#train1.dtypes

(750, 58)
(750, 23)

(250, 58)
(250, 23)


# Salvando Vetores Com Valores para substituir MIssing e Scale

## Substituindo Missing nas Variaveis Numericas

In [13]:
floatvar = pd.DataFrame(train1.dtypes == 'float64', columns = ['Float'])
floatvar = floatvar[floatvar['Float'] == True]
floatvar = floatvar.index.tolist()
floatvar

['duration_in_month',
 'credit_amount',
 'installment_as_income_perc',
 'present_res_since',
 'credits_this_bank']

In [14]:
train1[floatvar[0]].unique()

array([12., 48., 24., 18., 36.,  6., 20., 21., 15.,  9., 42., 45., 30.,
        7.,  8., 14., 22., 10.,  5., 60., 11., 16., 27., 33.,  4., 13.,
       54., 39., 72., 28., 47., 40.])

In [15]:
#train1[floatvar[1]].unique()

In [16]:
train1[floatvar[2]].unique()

array([4., 2., 3., 1.])

In [17]:
train1[floatvar[3]].unique()

array([2., 1., 4., 3.])

In [18]:
train1[floatvar[4]].unique()

array([3., 1., 2., 4.])

In [19]:
#Irei apenas padronizar as duas primeiras variaveis 'duration_in_month', 'credit_amount'
#Para as duas primeiras substituirei missing como mediana
#Para as tres ultimas substituirei missing por moda

In [20]:
Var = ['duration_in_month',  'credit_amount', 'installment_as_income_perc', 'present_res_since', 'credits_this_bank']

DICMedian = dicMedian(data = train1, variables = Var)

## Scale

In [21]:
DICMeanSD = dicMeanSd(data = train1, variables = Var)

In [22]:
with open(pathaux + 'Dictioniaries.pickle', 'wb') as f:
    dill.dump((DICMedian, DICMeanSD), f)

# Substituindo Missing e Fazendo Scaling em todas as bases!

In [23]:
#Treino
train2 = RemoveNA(data = train1, variables = Var, dic = DICMedian)
train = Scale(data = train2, variables = Var, dic = DICMeanSD)

In [24]:
#Teste
teste2 = RemoveNA(data = teste1, variables = Var, dic = DICMedian)
teste = Scale(data = teste2, variables = Var, dic = DICMeanSD)

In [25]:
#Verificando se ainda ficou alguma coluna com missing
train.isna().sum().sum()

0

In [26]:
teste.isna().sum().sum()

0

# Salvando Bases

In [27]:
train.to_pickle(pathparcial + 'Arquivo0 Treino.pkl')
teste.to_pickle(pathparcial + 'Arquivo0 Teste.pkl')

#train.to_excel(pathparcial + 'Arquivo0.xlsx', encoding = 'latin1', index = False)
sizes.to_excel(pathout + 'Tamanhos.xlsx', encoding = 'latin1', index = False)